In [1]:
# import modules
import copy
import time
from sys import path
from collections import defaultdict

In [2]:
import pandas as pd
from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [3]:
path.append('../code/src')

In [4]:
import TF_algo
from post_rec import PostRec
from inter_rec import InterRec
from evaluate import Evaluation as ev

In [5]:
# constants
# file path
input_path = "../data/"
output_path = '../result/'
rate_file = 'reduced_rating_data.csv'
attr_file = 'recipe_data.csv'

# user/item max
user_max = 1000
food_max = 5000

# const count
const_count = 3

In [21]:
# Dictionary keys
rs1 = 'w/o Constraint'
rs2 = 'Post-Single-'
rs3 = 'Inter-Single-'
val_r1 = 'RMSE'
val_r2 = 'nDCG'
val_t1 = 'train time(s)'
val_t2 = 'exec time(s)'

In [22]:
val_dict = {
    val_r1: 0,
    val_r2: 0,
    val_t1: 0,
    val_t2: 0
}

keys = [rs1]
keys = keys + [rs2 + str(i) for i in range (1, 4)]
keys = keys + [rs3 + str(i) for i in range (1, 4)]

result = {key: copy.deepcopy(val_dict) for key in keys}

In [8]:
# read previous result
result_df = pd.read_csv(output_path + 'evaluation.csv', index_col = 0).transpose()
result.update(result_df.to_dict())
result

{'w/o Constraint': {'RMSE': 0.7596431220184778,
  'nDCG': nan,
  'train time(s)': 0.2588422298431396,
  'exec time(s)': 14.873966217041016},
 'Post-Single-1': {'RMSE': 4.699165956844706,
  'nDCG': nan,
  'train time(s)': 0.260310411453247,
  'exec time(s)': 50.2104647954305},
 'Post-Single-2': {'RMSE': 4.699165956844706,
  'nDCG': nan,
  'train time(s)': 0.2589774131774902,
  'exec time(s)': 17.47806437810262},
 'Post-Single-3': {'RMSE': 0.8388547440770102,
  'nDCG': nan,
  'train time(s)': 0.264949083328247,
  'exec time(s)': 33.0610036055247},
 'Inter-Single-1': {'RMSE': 3.9436092463461008,
  'nDCG': nan,
  'train time(s)': 266.5387618541717,
  'exec time(s)': 10.34948476155599},
 'Inter-Single-2': {'RMSE': 2.918965609797331,
  'nDCG': nan,
  'train time(s)': 271.7621947924296,
  'exec time(s)': 17.99029239018758},
 'Inter-Single-3': {'RMSE': 0.8274846761344556,
  'nDCG': nan,
  'train time(s)': 339.3395245075226,
  'exec time(s)': 19.502410968144734}}

# Evaluate RS w/o Constraint
Check Suprise SVD performance with given rating data

In [9]:
# Source From: https://github.com/NicolasHug/Surprise/blob/master/examples/top_n_recommendations.py
def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    ret = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
        ret[int(uid)] = [int(i) for i, r in top_n[uid]]
    
    return ret

In [10]:
def run_SVD():    
    # get data
    reader = Reader(line_format='user item rating', sep=',', rating_scale=(0, 5))
    data = Dataset.load_from_file(input_path + rate_file, reader = reader)
    
    train_set, test_set = train_test_split(data, test_size=0.25)
    anti_set = train_set.build_anti_testset()
    
    # use SVD algorithm
    algo = SVD()
    
    # train
    start = time.time()
    algo.fit(train_set)
    t1 = time.time() - start
    
    # predict rating for test-set
    predict_test = algo.test(test_set)
    r1 = ev.calculate_rmse(predict_test)

    # get top-n for anti-test-set
    start = time.time()
    predict_full = algo.test(anti_set)
    top_n = get_top_n(predict_full)
    t2 = time.time() - start
    
    # predict nDCG for test-set
    rel_dict = defaultdict(list)
    for (u, i, r) in test_set:
        if r >= 4:
            rel_dict[int(u)].append(int(i))
    
    top_n_df = pd.DataFrame.from_dict(top_n, orient='index')
    top_n_df = top_n_df.reindex(columns=[x for x in range(0, 10)])
    r2 = ev.calculate_ndcg(rel_dict, top_n_df, 10)
    
    return r1, r2, t1, t2

In [24]:
r1, r2, t1, t2 = run_SVD()
result[rs1][val_r1] = r1
result[rs1][val_r2] = r2
result[rs1][val_t1] = t1
result[rs1][val_t2] = t2

# Evaluate Post-Rec w/ Single Type Constraint

In [26]:
def run_post(ctype, idx, save_result = True):    
    # PostRec applies constraint after the rating of each item is predicted
    rec = PostRec(input_path + rate_file, input_path + attr_file, 
                           input_path + 'const_' + str(ctype) + '.' + str(idx) + '.csv',
                            split = True)
    
    rec.get_data()  # get rating, attribute, recipe data
    
    # train with data
    start = time.time()
    rec.train()  
    t1 = time.time() - start
    
    # predict rating for test-set
    predict_test = rec.test_rmse()
    r1 = ev.calculate_rmse(predict_test)
       
    # get top-n for anti-test-set
    start = time.time()
    rec.test()
    top_n_df = rec.get_top_n()
    t2 = time.time() - start
    
    # calculate ndcg
    r2 = ev.calculate_ndcg(rec.get_rel(), top_n_df, 10)
    
    if save_result:       
        top_n_df.to_csv(output_path + 'PostRec_' + str(ctype) + '.' + str(idx) + '.csv')
        
    return r1, r2, t1, t2

In [27]:
for i in range (1, 4):
    r1_sum = 0
    r2_sum = 0
    t1_sum = 0
    t2_sum = 0
    for j in range (1, const_count + 1):
        r1, r2, t1, t2 = run_post(i, j, True)
        r1_sum = r1_sum + r1
        r2_sum = r2_sum + r2
        t1_sum = t1_sum + t1
        t2_sum = t2_sum + t2
        print('Const_'+str(i)+'.'+str(j)+" done")
        
    result[rs2+str(i)][val_r1] = r1_sum/const_count
    result[rs2+str(i)][val_r2] = r2_sum/const_count
    result[rs2+str(i)][val_t1] = t1_sum/const_count
    result[rs2+str(i)][val_t2] = t2_sum/const_count

Const_1.1 done
Const_1.2 done
Const_1.3 done
Const_2.1 done
Const_2.2 done
Const_2.3 done
Const_3.1 done
Const_3.2 done
Const_3.3 done


### Sample Dataframe

In [29]:
pd.read_csv(output_path + 'PostRec_1.1.csv', index_col = 0).head()

,0,1,2,3,4,5,6,7,8,9,i1,i2,nl
0,794,4104,25,828,1278,565,3295,2912,2851,425,1505,NaN,NaN
2,1633,3342,1224,3650,404,281,1493,3623,2996,458,7233,NaN,NaN
3,4692,4665,1856,4243,396,1970,1860,437,4302,870,3668,NaN,NaN
4,2250,2221,2031,335,630,1371,2769,3935,1157,1709,4574,NaN,NaN
5,4318,846,93,1287,1263,4586,4876,3571,925,4804,332,NaN,NaN


In [30]:
pd.read_csv(output_path + 'PostRec_2.1.csv', index_col = 0).head()

,0,1,2,3,4,5,6,7,8,9,i1,i2,nl
0,112,4738,42,2280,2943,2355,20,3258,4134,598,NaN,840,NaN
2,3102,276,740,621,3226,3602,2444,4290,4034,923,NaN,6270,NaN
3,648,4352,2949,1983,3459,1611,4713,1994,1921,1705,NaN,7449,NaN
4,3317,4642,2274,3314,125,193,648,3660,1932,2549,NaN,5010,NaN
5,4128,621,3472,484,3067,784,3155,1942,1611,4691,NaN,6270,NaN


In [31]:
pd.read_csv(output_path + 'PostRec_3.1.csv', index_col = 0).head()

,0,1,2,3,4,5,6,7,8,9,i1,i2,nl
0,1984,4152,193,569,1028,1151,2911,679,4945,647,NaN,NaN,"[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]"
2,391,40,112,4599,2958,2949,4913,3940,181,1464,NaN,NaN,"[132.3, 11.0, 39.0, 5.0, 4.0, 11.0, 5.0]"
3,1893,2072,2300,2280,338,3511,2274,315,3834,1221,NaN,NaN,"[29.4, 0.0, 23.0, 4.0, 0.0, 0.0, 2.0]"
4,2342,1854,983,3476,3775,112,3258,2178,647,3227,NaN,NaN,"[90.4, 6.0, 27.0, 4.0, 1.0, 11.0, 4.0]"
5,2280,2854,3581,3317,2274,4392,4943,3212,1628,4106,NaN,NaN,"[206.4, 0.0, 201.0, 0.0, 1.0, 0.0, 18.0]"


# RMSE of Inter-Rec w/ Single Type Constraint

In [32]:
def run_inter(ctype, idx, save_result = True):    
    # PostRec applies constraint after the rating of each item is predicted
    rec = InterRec(input_path + rate_file, input_path + attr_file, 
                           input_path + 'const_' + str(ctype) + '.' + str(idx) + '.csv', 
                            TF_algo.SVDtf(), split = True)
    
    rec.get_data()  # get rating, attribute, recipe data
    
    # train with data
    start = time.time()
    rec.train()  
    t1 = time.time() - start
    
    # predict rating for test-set
    predict_test = rec.test_rmse()
    r1 = ev.calculate_rmse(predict_test)
    
    # get top-n for anti-test-set
    start = time.time()
    rec.test()
    top_n_df = rec.get_top_n()
    t2 = time.time() - start

    # calculate nDCG
    r2 = ev.calculate_ndcg(rec.get_rel(), top_n_df, 10)
    
    if save_result:       
        top_n_df.to_csv(output_path + 'InterRec_' + str(ctype) + '.' + str(idx) + '.csv')
        
    return r1, r2, t1, t2

In [33]:
for i in range (1, 4):
    r1_sum = 0
    r2_sum = 0
    t1_sum = 0
    t2_sum = 0
    for j in range (1, const_count + 1):
        r1, r2, t1, t2 = run_inter(i, j, True)
        r1_sum = r1_sum + r1
        r2_sum = r2_sum + r2
        t1_sum = t1_sum + t1
        t2_sum = t2_sum + t2
        print('Const_'+str(i)+'.'+str(j)+" done")
        
    result[rs3+str(i)][val_r1] = r1_sum/const_count
    result[rs3+str(i)][val_r2] = r2_sum/const_count
    result[rs3+str(i)][val_t1] = t1_sum/const_count
    result[rs3+str(i)][val_t2] = t2_sum/const_count

Const_1.1 done
Const_1.2 done
Const_1.3 done
Const_2.1 done
Const_2.2 done
Const_2.3 done
Const_3.1 done
Const_3.2 done
Const_3.3 done


In [34]:
pd.read_csv(output_path + 'InterRec_1.1.csv', index_col = 0).head()

,0,1,2,3,4,5,6,7,8,9,i1,i2,nl
309,3802,3386,1056,2340,421,2551,239,181,2609,321,6426,NaN,NaN
564,3146,1262,4032,3018,2639,2525,3880,1815,3342,1341,2320,NaN,NaN
560,3198,4895,1957,4202,4318,4551,3342,3623,3901,3781,7233,NaN,NaN
25,193,505,983,2355,4936,2903,4603,4503,712,4986,5006,NaN,NaN
873,4683,1317,269,3039,3145,239,1557,874,3475,4482,6276,NaN,NaN


In [35]:
pd.read_csv(output_path + 'InterRec_2.1.csv', index_col = 0).head()

,0,1,2,3,4,5,6,7,8,9,i1,i2,nl
309,4005,4105,4195,4570,2072,1611,1751,4682,4890,3459,NaN,6270,NaN
564,361,2274,193,112,740,4226,4874,2970,3057,4683,NaN,332,NaN
560,4270,2136,1984,2355,1437,882,14,1829,609,828,NaN,5180,NaN
25,505,1690,361,1337,945,2454,193,4770,397,4683,NaN,332,NaN
873,3510,2274,3459,4652,265,4839,1829,4274,836,820,NaN,6270,NaN


In [36]:
pd.read_csv(output_path + 'InterRec_3.1.csv', index_col = 0).head()

,0,1,2,3,4,5,6,7,8,9,i1,i2,nl
309,4661,21,2988,4658,3743,2645,1287,3612,495,907,NaN,NaN,"[245.1, 7.0, 7.0, 28.0, 83.0, 6.0, 1.0]"
564,4778,677,617,1169,276,4789,3102,1984,4783,708,NaN,NaN,"[35.9, 1.0, 5.0, 6.0, 5.0, 3.0, 1.0]"
560,146,1690,855,109,1736,1192,2988,3238,1827,4745,NaN,NaN,"[9.2, 0.0, 2.0, 1.0, 2.0, 0.0, 0.0]"
25,4945,1531,2608,960,2179,1287,4683,1653,2841,3345,NaN,NaN,"[32.2, 5.0, 0.0, 0.0, 0.0, 2.0, 0.0]"
873,2549,3285,2274,4392,4457,2468,4702,2280,83,3865,NaN,NaN,"[61.8, 0.0, 42.0, 7.0, 0.0, 0.0, 5.0]"


# Result

In [37]:
result_df = pd.DataFrame.from_dict(result, orient='index', columns=['RMSE', 'nDCG', 'train time(s)', 'exec time(s)'])
result_df.to_csv(output_path + 'evaluation.csv')
result_df

,RMSE,nDCG,train time(s),exec time(s)
w/o Constraint,0.843047,0.006460,0.271575,14.656277
Post-Single-1,4.699166,0.069172,0.261433,48.989032
Post-Single-2,4.699166,0.009632,0.263638,17.429793
Post-Single-3,0.838979,0.020314,0.261169,32.935676
Inter-Single-1,3.943666,0.059580,253.569261,9.680154
Inter-Single-2,2.919561,0.014151,257.824929,17.210232
Inter-Single-3,0.828984,0.021002,332.494968,20.116116
